<a href="https://colab.research.google.com/github/PulyaPolya/Fashion_mnist/blob/main/New_oracle_32_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALL

In [ ]:
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
import json
from sklearn.utils import shuffle
from keras.datasets import mnist
from keras.datasets import fashion_mnist
from scipy.ndimage.interpolation import shift
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback


def edit_data(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_train = x_train / 255.0

    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
    x_test = x_test / 255.0
    y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
    y_test = tf.one_hot(y_test.astype(np.int32), depth=10)
    return x_train, y_train, x_test, y_test

<ipython-input-1-b9b5c68dfe3e>:9: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [ ]:
def shift_image(image, dx='NAN', dy='NAN'):
    if dx =='NAN' or dy =='NAN':
        dx, dy = random.choice([(1, 0), (-1, 0), (0, 1), (0, -1), (0, 0)])
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image
def shift_image_np(x_batch, dx ='nan', dy ='nan'):
    if dx == 'nan'and dy == 'nan':
        dx, dy = random.choice([(1,0), (-1,0), (0,1), (0,-1), (0,0)])
    x_shifted = np.pad(x_batch, ((abs(dx), 0), (abs(dy), 0)), mode='constant')
    x_shifted = np.roll(x_shifted, (dx, dy), axis=(0, 1))[abs(dx):, abs(dy):]
    return x_shifted

In [ ]:
"""Functions for downloading and reading Oracle-MNIST data."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import gzip
import os
import tensorflow.python.platform
import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf


def _read32(bytestream):
  dt = numpy.dtype(numpy.uint32).newbyteorder('>')
  return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]

def extract_images(filename):
  """Extract the images into a 4D uint8 numpy array [index, y, x, depth]."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    magic = _read32(bytestream)
    if magic != 2051:
      raise ValueError('Invalid magic number %d in Oracle-MNIST image file: %s' % (magic, filename))
    num_images = _read32(bytestream)
    rows = _read32(bytestream)
    cols = _read32(bytestream)
    buf = bytestream.read(rows * cols * num_images)
    data = numpy.frombuffer(buf, dtype=numpy.uint8)
    data = data.reshape(num_images, rows, cols, 1)
    return data

def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

def extract_labels(filename, one_hot=False):
  """Extract the labels into a 1D uint8 numpy array [index]."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    magic = _read32(bytestream)
    if magic != 2049:
      raise ValueError(
          'Invalid magic number %d in Oracle-MNIST label file: %s' % (magic, filename))
    num_items = _read32(bytestream)
    buf = bytestream.read(num_items)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8)
    if one_hot:
      return dense_to_one_hot(labels)
    return labels

class DataSet(object):
  def __init__(self, images, labels, fake_data=False, one_hot=False, dtype=tf.float32):
    """Construct a DataSet.
    one_hot arg is used only if fake_data is true.  `dtype` can be either
    `uint8` to leave the input as `[0, 255]`, or `float32` to rescale into
    `[0, 1]`.
    """
    dtype = tf.as_dtype(dtype).base_dtype
    if dtype not in (tf.uint8, tf.float32):
      raise TypeError('Invalid image dtype %r, expected uint8 or float32' % dtype)
    if fake_data:
      self._num_examples = 10000
      self.one_hot = one_hot
    else:
      assert images.shape[0] == labels.shape[0], ('images.shape: %s labels.shape: %s' % (images.shape, labels.shape))
      self._num_examples = images.shape[0]
      # Convert shape from [num examples, rows, columns, depth]
      # to [num examples, rows*columns] (assuming depth == 1)
      assert images.shape[3] == 1
      images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])
      if dtype == tf.float32:
        # Convert from [0, 255] -> [0.0, 1.0].
        images = images.astype(numpy.float32)
        images = numpy.multiply(images, 1.0 / 255.0)
    self._images = images
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0
  @property
  def images(self):
    return self._images
  @property
  def labels(self):
    return self._labels
  @property
  def num_examples(self):
    return self._num_examples
  @property
  def epochs_completed(self):
    return self._epochs_completed
  def next_batch(self, batch_size, fake_data=False):
    """Return the next `batch_size` examples from this data set."""
    if fake_data:
      fake_image = [1] * 784
      if self.one_hot:
        fake_label = [1] + [0] * 9
      else:
        fake_label = 0
      return [fake_image for _ in xrange(batch_size)], [fake_label for _ in xrange(batch_size)]
    start = self._index_in_epoch
    self._index_in_epoch += batch_size
    if self._index_in_epoch > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Shuffle the data
      perm = numpy.arange(self._num_examples)
      numpy.random.shuffle(perm)
      self._images = self._images[perm]
      self._labels = self._labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch
    return self._images[start:end], self._labels[start:end]

def read_data_sets(train_dir, one_hot=False, valid_num=0, dtype=tf.float32):
  class DataSets(object):
    pass
  data_sets = DataSets()

  TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
  TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
  TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
  TEST_LABELS = 't10k-labels-idx1-ubyte.gz'

  x_train = extract_images(filename=os.path.join(train_dir, TRAIN_IMAGES))
  y_train = extract_labels(filename=os.path.join(train_dir, TRAIN_LABELS), one_hot=one_hot)
  x_test = extract_images(filename=os.path.join(train_dir, TEST_IMAGES))
  y_test = extract_labels(filename=os.path.join(train_dir, TEST_LABELS), one_hot=one_hot)
  # data_sets.test = DataSet(test_images, test_labels, dtype=dtype)
  #
  # if valid_num != 0:
  #   validation_images = train_images[:valid_num]
  #   validation_labels = train_labels[:valid_num]
  #   train_images = train_images[valid_num:]
  #   train_labels = train_labels[valid_num:]
  #   data_sets.train = DataSet(train_images, train_labels, dtype=dtype)
  #   data_sets.validation = DataSet(validation_images, validation_labels, dtype=dtype)
  # else:
  #   data_sets.train = DataSet(train_images, train_labels, dtype=dtype)

  return x_train, y_train, x_test, y_test



In [ ]:
x_train, y_train, x_test, y_test = read_data_sets('/content/drive/MyDrive/experiments/oracle-mnist', one_hot=False,
                                                                                     valid_num=0, dtype=tf.float32)


Extracting /content/drive/MyDrive/experiments/oracle-mnist/train-images-idx3-ubyte.gz
Extracting /content/drive/MyDrive/experiments/oracle-mnist/train-labels-idx1-ubyte.gz
Extracting /content/drive/MyDrive/experiments/oracle-mnist/t10k-images-idx3-ubyte.gz
Extracting /content/drive/MyDrive/experiments/oracle-mnist/t10k-labels-idx1-ubyte.gz


In [ ]:
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.callbacks import ModelCheckpoint, TensorBoard
import time
from tensorflow.keras import layers
from keras.callbacks import LambdaCallback
import numpy as np

class Epoch_Tracker:
  def __init__(self):
     self.epoch = 0
     self.change = True
  def increase(self):
    self.epoch +=1
    self.change = True

epoch_track = Epoch_Tracker()

start = time.time()
x_train, y_train, x_test, y_test = edit_data(x_train, y_train, x_test, y_test)

size_data = x_train.shape[0]
batch_size = 32
num_classes = 10
epochs =15

NAME = "Bayes"

def random_invert_img(x):
  #print(epoch_track.epoch)
  if epoch_track.epoch >= epochs:
      return x
  x_temp = x.numpy()
  x_temp = x_temp.reshape(x_temp.shape[0], 28,28)
  x_shifted = []
  for image in x_temp:
      x_shifted.append(shift_image_np(image))
  x_shifted = np.array(x_shifted)

  x_result = x_shifted.reshape(x_temp.shape[0],28,28,1)
  return x_result
def random_invert():
  return layers.Lambda(lambda x: random_invert_img(x))

random_invert = random_invert()
class RandomInvert(layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def call(self, x):
    return random_invert_img(x)





# Bayes

In [ ]:
x_train.shape

(27222, 28, 28, 1)

In [ ]:
tf.random.set_seed(1234) #oracle32 bayes fold 1
input_shape = (28, 28, 1)
conv1 = 77
conv2 = 40
cov3 = 80
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 7
drop1 = 4
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/bayes/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/bayes/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 bayes fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


766/766 [==============================] - ETA: 0s - loss: 0.9597 - acc: 0.6689
Epoch 1: val_acc improved from -inf to 0.84686, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold1/cnn_saved_model/weights-improvement--01-0.46.hdf5
766/766 [==============================] - 40s 46ms/step - loss: 0.9597 - acc: 0.6689 - val_loss: 0.4551 - val_acc: 0.8469
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.4322 - acc: 0.8555
Epoch 2: val_acc improved from 0.84686 to 0.90305, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold1/cnn_saved_model/weights-improvement--02-0.31.hdf5
766/766 [==============================] - 35s 46ms/step - loss: 0.4324 - acc: 0.8555 - val_loss: 0.3109 - val_acc: 0.9030
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.3354 - acc: 0.8907
Epoch 3: val_acc improved from 0.90305 to 0.90892, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold1/cnn_saved_mode

In [ ]:
tf.random.set_seed(1234) #oracle32 bayes fold 2
input_shape = (28, 28, 1)
conv1 = 140
conv2 = 40
cov3 = 80
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 15
drop1 = 6
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/bayes/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/bayes/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 bayes fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8836 - acc: 0.6923
Epoch 1: val_acc improved from -inf to 0.85751, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold2/cnn_saved_model/weights-improvement--01-0.42.hdf5
766/766 [==============================] - 31s 38ms/step - loss: 0.8836 - acc: 0.6923 - val_loss: 0.4247 - val_acc: 0.8575
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3950 - acc: 0.8667
Epoch 2: val_acc improved from 0.85751 to 0.91039, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold2/cnn_saved_model/weights-improvement--02-0.28.hdf5
766/766 [==============================] - 28s 37ms/step - loss: 0.3951 - acc: 0.8666 - val_loss: 0.2769 - val_acc: 0.9104
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.3192 - acc: 0.8925
Epoch 3: val_acc did not improve from 0.91039
766/766 [==============================] - 28s 37ms/step - loss: 0.3193 - acc: 0.892

In [ ]:
tf.random.set_seed(1234) #oracle32 bayes fold 3
input_shape = (28, 28, 1)
conv1 = 140
conv2 = 100
cov3 = 80
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 9
drop1 = 4
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/bayes/fold3/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/bayes/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 bayes fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8050 - acc: 0.7235
Epoch 1: val_acc improved from -inf to 0.86889, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold3/cnn_saved_model/weights-improvement--01-0.39.hdf5
766/766 [==============================] - 31s 38ms/step - loss: 0.8050 - acc: 0.7235 - val_loss: 0.3910 - val_acc: 0.8689
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3437 - acc: 0.8853
Epoch 2: val_acc improved from 0.86889 to 0.90966, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold3/cnn_saved_model/weights-improvement--02-0.27.hdf5
766/766 [==============================] - 29s 38ms/step - loss: 0.3438 - acc: 0.8853 - val_loss: 0.2699 - val_acc: 0.9097
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.2552 - acc: 0.9154
Epoch 3: val_acc improved from 0.90966 to 0.91884, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold3/cnn

In [ ]:
tf.random.set_seed(1234) #oracle32 bayes fold 4
input_shape = (28, 28, 1)
conv1 = 140
conv2 = 100
cov3 = 32
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 15
drop1 = 6
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/bayes/fold4/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/bayes/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 bayes fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8691 - acc: 0.6951
Epoch 1: val_acc improved from -inf to 0.87404, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold4/cnn_saved_model/weights-improvement--01-0.37.hdf5
766/766 [==============================] - 36s 44ms/step - loss: 0.8691 - acc: 0.6951 - val_loss: 0.3747 - val_acc: 0.8740
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3889 - acc: 0.8715
Epoch 2: val_acc improved from 0.87404 to 0.90929, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold4/cnn_saved_model/weights-improvement--02-0.27.hdf5
766/766 [==============================] - 33s 42ms/step - loss: 0.3889 - acc: 0.8715 - val_loss: 0.2680 - val_acc: 0.9093
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.3102 - acc: 0.8956
Epoch 3: val_acc improved from 0.90929 to 0.91957, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold4/cnn

In [ ]:
tf.random.set_seed(1234) #oracle32 bayes fold 5
input_shape = (28, 28, 1)
conv1 = 140
conv2 = 100
cov3 = 59
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 15
drop1 = 6
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/bayes/fold5/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/bayes/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 bayes fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8000 - acc: 0.7213
Epoch 1: val_acc improved from -inf to 0.86889, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold5/cnn_saved_model/weights-improvement--01-0.38.hdf5
766/766 [==============================] - 37s 46ms/step - loss: 0.8000 - acc: 0.7213 - val_loss: 0.3811 - val_acc: 0.8689
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3520 - acc: 0.8815
Epoch 2: val_acc improved from 0.86889 to 0.91443, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold5/cnn_saved_model/weights-improvement--02-0.26.hdf5
766/766 [==============================] - 35s 45ms/step - loss: 0.3520 - acc: 0.8815 - val_loss: 0.2559 - val_acc: 0.9144
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2806 - acc: 0.9057
Epoch 3: val_acc improved from 0.91443 to 0.92876, saving model to /content/drive/MyDrive/experiments/oracle32_new/bayes/fold5/cnn

# Random

In [ ]:
tf.random.set_seed(1234) #oracle32 random fold 1
input_shape = (28, 28, 1)
conv1 = 60
conv2 = 71
cov3 = 44
l_rate = 7
kernel1 = 3
kernel2=3
kernel3 = 13
drop1 = 4
drop2 = 6


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/random/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/random/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 random fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7391 - acc: 0.7442
Epoch 1: val_acc improved from -inf to 0.88248, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold1/cnn_saved_model/weights-improvement--01-0.34.hdf5
766/766 [==============================] - 37s 44ms/step - loss: 0.7391 - acc: 0.7442 - val_loss: 0.3449 - val_acc: 0.8825
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3337 - acc: 0.8882
Epoch 2: val_acc improved from 0.88248 to 0.91186, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold1/cnn_saved_model/weights-improvement--02-0.26.hdf5
766/766 [==============================] - 31s 40ms/step - loss: 0.3337 - acc: 0.8882 - val_loss: 0.2646 - val_acc: 0.9119
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2577 - acc: 0.9114
Epoch 3: val_acc improved from 0.91186 to 0.92692, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold1/

In [ ]:
tf.random.set_seed(1234) #oracle32 random fold 2
input_shape = (28, 28, 1)
conv1 = 73
conv2 = 56
cov3 = 77
l_rate = 12
kernel1 = 3
kernel2=5
kernel3 = 5
drop1 = 4
drop2 = 6


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new2/random/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/random/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 random fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7221 - acc: 0.7528
Epoch 1: val_acc improved from -inf to 0.88542, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold2/cnn_saved_model/weights-improvement--01-0.36.hdf5
766/766 [==============================] - 32s 39ms/step - loss: 0.7221 - acc: 0.7528 - val_loss: 0.3583 - val_acc: 0.8854
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3421 - acc: 0.8842
Epoch 2: val_acc improved from 0.88542 to 0.90305, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold2/cnn_saved_model/weights-improvement--02-0.28.hdf5
766/766 [==============================] - 29s 37ms/step - loss: 0.3421 - acc: 0.8842 - val_loss: 0.2792 - val_acc: 0.9030
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2837 - acc: 0.9037
Epoch 3: val_acc improved from 0.90305 to 0.92655, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold2/

In [ ]:
tf.random.set_seed(1234) #oracle32 random fold 3
input_shape = (28, 28, 1)
conv1 = 48
conv2 = 73
cov3 = 59
l_rate = 7
kernel1 = 3
kernel2=5
kernel3 = 9
drop1 = 4
drop2 = 6


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/random/fold3/random_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/random/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 random fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7078 - acc: 0.7550
Epoch 1: val_acc improved from -inf to 0.89056, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold3/cnn_saved_model/weights-improvement--01-0.32.hdf5
766/766 [==============================] - 32s 39ms/step - loss: 0.7078 - acc: 0.7550 - val_loss: 0.3198 - val_acc: 0.8906
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3122 - acc: 0.8962
Epoch 2: val_acc improved from 0.89056 to 0.91774, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold3/cnn_saved_model/weights-improvement--02-0.25.hdf5
766/766 [==============================] - 30s 39ms/step - loss: 0.3122 - acc: 0.8963 - val_loss: 0.2465 - val_acc: 0.9177
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2473 - acc: 0.9168
Epoch 3: val_acc improved from 0.91774 to 0.92508, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold3/

In [ ]:
tf.random.set_seed(1234) #oracle32 random fold 4
input_shape = (28, 28, 1)
conv1 = 98
conv2 = 84
cov3 = 70
l_rate = 5
kernel1 = 5
kernel2=5
kernel3 = 7
drop1 = 6
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/random/fold4/random_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/random/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 random fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8238 - acc: 0.7185
Epoch 1: val_acc improved from -inf to 0.86522, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold4/cnn_saved_model/weights-improvement--01-0.39.hdf5
766/766 [==============================] - 34s 41ms/step - loss: 0.8238 - acc: 0.7185 - val_loss: 0.3852 - val_acc: 0.8652
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3517 - acc: 0.8817
Epoch 2: val_acc improved from 0.86522 to 0.90599, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold4/cnn_saved_model/weights-improvement--02-0.29.hdf5
766/766 [==============================] - 30s 39ms/step - loss: 0.3518 - acc: 0.8816 - val_loss: 0.2866 - val_acc: 0.9060
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2727 - acc: 0.9094
Epoch 3: val_acc improved from 0.90599 to 0.91260, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold4/

In [ ]:
tf.random.set_seed(1234) #oracle32 random fold 5
input_shape = (28, 28, 1)
conv1 = 116
conv2 = 97
cov3 = 55
l_rate = 7
kernel1 = 5
kernel2=5
kernel3 = 11
drop1 = 6
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/random/fold5/random_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/random/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 random fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7248 - acc: 0.7537
Epoch 1: val_acc improved from -inf to 0.87404, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold5/cnn_saved_model/weights-improvement--01-0.38.hdf5
766/766 [==============================] - 36s 43ms/step - loss: 0.7248 - acc: 0.7537 - val_loss: 0.3795 - val_acc: 0.8740
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3295 - acc: 0.8905
Epoch 2: val_acc improved from 0.87404 to 0.90635, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold5/cnn_saved_model/weights-improvement--02-0.27.hdf5
766/766 [==============================] - 32s 42ms/step - loss: 0.3295 - acc: 0.8905 - val_loss: 0.2658 - val_acc: 0.9064
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.2682 - acc: 0.9102
Epoch 3: val_acc improved from 0.90635 to 0.91921, saving model to /content/drive/MyDrive/experiments/oracle32_new/random/fold5/

# Hyperband

In [ ]:
tf.random.set_seed(1234) #hyperband random fold 1
input_shape = (28, 28, 1)
conv1 = 66
conv2 = 67
cov3 = 70
l_rate = 9
kernel1 = 3
kernel2=3
kernel3 = 7
drop1 = 3
drop2 = 4


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold1/hyperband_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 hyperband fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.6776 - acc: 0.7697
Epoch 1: val_acc improved from -inf to 0.88799, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold1/cnn_saved_model/weights-improvement--01-0.33.hdf5
766/766 [==============================] - 30s 37ms/step - loss: 0.6776 - acc: 0.7697 - val_loss: 0.3341 - val_acc: 0.8880
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.2864 - acc: 0.9047
Epoch 2: val_acc improved from 0.88799 to 0.92288, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold1/cnn_saved_model/weights-improvement--02-0.24.hdf5
766/766 [==============================] - 28s 37ms/step - loss: 0.2864 - acc: 0.9047 - val_loss: 0.2379 - val_acc: 0.9229
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2169 - acc: 0.9266
Epoch 3: val_acc improved from 0.92288 to 0.92839, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperba

In [ ]:
tf.random.set_seed(1234) #hyperband random fold 2
input_shape = (28, 28, 1)
conv1 = 62
conv2 = 44
cov3 = 45
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 13
drop1 = 4
drop2 = 6


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold2/hyperband_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 hyperband fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8216 - acc: 0.7183
Epoch 1: val_acc improved from -inf to 0.86485, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold2/cnn_saved_model/weights-improvement--01-0.40.hdf5
766/766 [==============================] - 35s 42ms/step - loss: 0.8216 - acc: 0.7183 - val_loss: 0.3971 - val_acc: 0.8649
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3505 - acc: 0.8829
Epoch 2: val_acc improved from 0.86485 to 0.91407, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold2/cnn_saved_model/weights-improvement--02-0.27.hdf5
766/766 [==============================] - 31s 41ms/step - loss: 0.3505 - acc: 0.8829 - val_loss: 0.2675 - val_acc: 0.9141
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2717 - acc: 0.9067
Epoch 3: val_acc improved from 0.91407 to 0.92068, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperba

In [ ]:
tf.random.set_seed(1234) #hyperband random fold 3
input_shape = (28, 28, 1)
conv1 = 71
conv2 = 78
cov3 = 33
l_rate = 5
kernel1 = 3
kernel2=5
kernel3 = 9
drop1 = 5
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold3/hyperband_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 hyperband fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8982 - acc: 0.6893
Epoch 1: val_acc improved from -inf to 0.85751, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold3/cnn_saved_model/weights-improvement--01-0.43.hdf5
766/766 [==============================] - 31s 38ms/step - loss: 0.8982 - acc: 0.6893 - val_loss: 0.4267 - val_acc: 0.8575
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3717 - acc: 0.8756
Epoch 2: val_acc improved from 0.85751 to 0.90158, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold3/cnn_saved_model/weights-improvement--02-0.29.hdf5
766/766 [==============================] - 28s 37ms/step - loss: 0.3718 - acc: 0.8756 - val_loss: 0.2932 - val_acc: 0.9016
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2959 - acc: 0.9000
Epoch 3: val_acc improved from 0.90158 to 0.92031, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperba

In [ ]:
tf.random.set_seed(1234) #hyperband random fold 4
input_shape = (28, 28, 1)
conv1 = 75
conv2 = 42
cov3 = 44
l_rate = 8
kernel1 = 3
kernel2=5
kernel3 = 13
drop1 = 4
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold4/hyperband_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 hyperband fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7018 - acc: 0.7585
Epoch 1: val_acc improved from -inf to 0.89423, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold4/cnn_saved_model/weights-improvement--01-0.32.hdf5
766/766 [==============================] - 32s 39ms/step - loss: 0.7018 - acc: 0.7585 - val_loss: 0.3220 - val_acc: 0.8942
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3142 - acc: 0.8952
Epoch 2: val_acc improved from 0.89423 to 0.91443, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold4/cnn_saved_model/weights-improvement--02-0.25.hdf5
766/766 [==============================] - 29s 38ms/step - loss: 0.3142 - acc: 0.8951 - val_loss: 0.2454 - val_acc: 0.9144
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.2480 - acc: 0.9164
Epoch 3: val_acc improved from 0.91443 to 0.92104, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperba

In [ ]:
tf.random.set_seed(1234) #hyperband random fold 5
input_shape = (28, 28, 1)
conv1 = 121
conv2 = 84
cov3 = 75
l_rate = 8
kernel1 = 3
kernel2=5
kernel3 = 11
drop1 = 6
drop2 = 4


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold5/hyperband_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/hyperband/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 hyperband fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.6947 - acc: 0.7599
Epoch 1: val_acc improved from -inf to 0.88542, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold5/cnn_saved_model/weights-improvement--01-0.33.hdf5
766/766 [==============================] - 31s 38ms/step - loss: 0.6947 - acc: 0.7599 - val_loss: 0.3332 - val_acc: 0.8854
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3190 - acc: 0.8946
Epoch 2: val_acc improved from 0.88542 to 0.91664, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperband/fold5/cnn_saved_model/weights-improvement--02-0.25.hdf5
766/766 [==============================] - 28s 37ms/step - loss: 0.3190 - acc: 0.8946 - val_loss: 0.2504 - val_acc: 0.9166
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.2545 - acc: 0.9158
Epoch 3: val_acc improved from 0.91664 to 0.92508, saving model to /content/drive/MyDrive/experiments/oracle32_new/hyperba

# SH

In [ ]:
tf.random.set_seed(1234) #oracle32 sh  fold 1
input_shape = (28, 28, 1)
conv1 = 80
conv2 = 76
cov3 = 46
l_rate = 6
kernel1 = 3
kernel2=7
kernel3 = 13
drop1 = 5
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/sh/fold1/sh_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/sh/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 sh fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.6819 - acc: 0.7646
Epoch 1: val_acc improved from -inf to 0.88285, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold1/cnn_saved_model/weights-improvement--01-0.35.hdf5
766/766 [==============================] - 38s 46ms/step - loss: 0.6819 - acc: 0.7646 - val_loss: 0.3501 - val_acc: 0.8828
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.2918 - acc: 0.9042
Epoch 2: val_acc improved from 0.88285 to 0.92068, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold1/cnn_saved_model/weights-improvement--02-0.24.hdf5
766/766 [==============================] - 34s 44ms/step - loss: 0.2918 - acc: 0.9042 - val_loss: 0.2422 - val_acc: 0.9207
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2221 - acc: 0.9257
Epoch 3: val_acc improved from 0.92068 to 0.92472, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold1/cnn_saved_mo

In [ ]:
tf.random.set_seed(1234) #oracle32 sh fold 2
input_shape = (28, 28, 1)
conv1 = 61
conv2 = 51
cov3 = 76
l_rate = 8
kernel1 = 3
kernel2=5
kernel3 = 13
drop1 = 4
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/sh/fold2/sh_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/sh/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 sh fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.6799 - acc: 0.7684
Epoch 1: val_acc improved from -inf to 0.88395, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold2/cnn_saved_model/weights-improvement--01-0.34.hdf5
766/766 [==============================] - 31s 37ms/step - loss: 0.6799 - acc: 0.7684 - val_loss: 0.3445 - val_acc: 0.8840
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.2962 - acc: 0.9019
Epoch 2: val_acc improved from 0.88395 to 0.92655, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold2/cnn_saved_model/weights-improvement--02-0.23.hdf5
766/766 [==============================] - 28s 37ms/step - loss: 0.2964 - acc: 0.9019 - val_loss: 0.2342 - val_acc: 0.9266
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2223 - acc: 0.9257
Epoch 3: val_acc did not improve from 0.92655
766/766 [==============================] - 28s 37ms/step - loss: 0.2223 - acc: 0.9258 - va

In [ ]:
tf.random.set_seed(1234) #oracle32 sh fold 3
input_shape = (28, 28, 1)
conv1 = 92
conv2 = 82
cov3 = 74
l_rate =11
kernel1 = 3
kernel2=5
kernel3 = 7
drop1 = 5
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/sh/fold3/sh_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/sh/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 sh fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.6257 - acc: 0.7877
Epoch 1: val_acc improved from -inf to 0.89644, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold3/cnn_saved_model/weights-improvement--01-0.32.hdf5
766/766 [==============================] - 34s 42ms/step - loss: 0.6257 - acc: 0.7877 - val_loss: 0.3235 - val_acc: 0.8964
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.2942 - acc: 0.9029
Epoch 2: val_acc improved from 0.89644 to 0.91076, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold3/cnn_saved_model/weights-improvement--02-0.26.hdf5
766/766 [==============================] - 31s 41ms/step - loss: 0.2942 - acc: 0.9029 - val_loss: 0.2632 - val_acc: 0.9108
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.2359 - acc: 0.9217
Epoch 3: val_acc improved from 0.91076 to 0.92288, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold3/cnn_saved_mo

In [ ]:
tf.random.set_seed(1234) #oracle32 sh fold 4
input_shape = (28, 28, 1)
conv1 = 53
conv2 = 54
cov3 = 78
l_rate =6
kernel1 = 3
kernel2=5
kernel3 = 13
drop1 = 5
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/sh/fold4/sh_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/sh/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 sh fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


766/766 [==============================] - ETA: 0s - loss: 0.8229 - acc: 0.7135
Epoch 1: val_acc improved from -inf to 0.87404, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold4/cnn_saved_model/weights-improvement--01-0.39.hdf5
766/766 [==============================] - 40s 38ms/step - loss: 0.8229 - acc: 0.7135 - val_loss: 0.3926 - val_acc: 0.8740
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3788 - acc: 0.8726
Epoch 2: val_acc improved from 0.87404 to 0.90966, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold4/cnn_saved_model/weights-improvement--02-0.28.hdf5
766/766 [==============================] - 29s 38ms/step - loss: 0.3788 - acc: 0.8726 - val_loss: 0.2841 - val_acc: 0.9097
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2968 - acc: 0.9004
Epoch 3: val_acc improved from 0.90966 to 0.91700, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold4/cnn_saved_model/weights

In [ ]:
tf.random.set_seed(1234) #oracle32 sh fold 5
input_shape = (28, 28, 1)
conv1 = 93
conv2 = 90
cov3 = 62
l_rate =6
kernel1 = 5
kernel2=5
kernel3 = 9
drop1 = 6
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/sh/fold5/sh_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/sh/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 sh fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8280 - acc: 0.7140
Epoch 1: val_acc improved from -inf to 0.86963, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold5/cnn_saved_model/weights-improvement--01-0.38.hdf5
766/766 [==============================] - 29s 35ms/step - loss: 0.8280 - acc: 0.7140 - val_loss: 0.3780 - val_acc: 0.8696
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3587 - acc: 0.8815
Epoch 2: val_acc improved from 0.86963 to 0.90672, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold5/cnn_saved_model/weights-improvement--02-0.28.hdf5
766/766 [==============================] - 26s 35ms/step - loss: 0.3586 - acc: 0.8815 - val_loss: 0.2777 - val_acc: 0.9067
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2805 - acc: 0.9055
Epoch 3: val_acc improved from 0.90672 to 0.91627, saving model to /content/drive/MyDrive/experiments/oracle32_new/sh/fold5/cnn_saved_mo

# Evolution

In [ ]:
tf.random.set_seed(1234) #oracle32 evo fold 1
input_shape = (28, 28, 1)
conv1 = 83
conv2 = 73
cov3 = 39
l_rate =6
kernel1 = 3
kernel2=7
kernel3 = 11
drop1 = 4
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/evo/fold1/evo_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/evo/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 evo fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7398 - acc: 0.7440
Epoch 1: val_acc improved from -inf to 0.87110, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold1/cnn_saved_model/weights-improvement--01-0.38.hdf5
766/766 [==============================] - 35s 43ms/step - loss: 0.7398 - acc: 0.7440 - val_loss: 0.3792 - val_acc: 0.8711
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3079 - acc: 0.8967
Epoch 2: val_acc improved from 0.87110 to 0.90892, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold1/cnn_saved_model/weights-improvement--02-0.26.hdf5
766/766 [==============================] - 32s 42ms/step - loss: 0.3080 - acc: 0.8967 - val_loss: 0.2643 - val_acc: 0.9089
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2390 - acc: 0.9205
Epoch 3: val_acc improved from 0.90892 to 0.92104, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold1/cnn_saved

In [ ]:
tf.random.set_seed(1234) #oracle32 evo fold 2
input_shape = (28, 28, 1)
conv1 = 94
conv2 = 100
cov3 =59
l_rate =5
kernel1 = 3
kernel2=7
kernel3 = 11
drop1 = 6
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/evo/fold2/evo_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/evo/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 evo fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8169 - acc: 0.7168
Epoch 1: val_acc improved from -inf to 0.86632, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold2/cnn_saved_model/weights-improvement--01-0.39.hdf5
766/766 [==============================] - 34s 41ms/step - loss: 0.8169 - acc: 0.7168 - val_loss: 0.3913 - val_acc: 0.8663
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3403 - acc: 0.8862
Epoch 2: val_acc improved from 0.86632 to 0.91370, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold2/cnn_saved_model/weights-improvement--02-0.26.hdf5
766/766 [==============================] - 28s 36ms/step - loss: 0.3403 - acc: 0.8862 - val_loss: 0.2638 - val_acc: 0.9137
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2686 - acc: 0.9108
Epoch 3: val_acc improved from 0.91370 to 0.92068, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold2/cnn_saved

In [ ]:
tf.random.set_seed(1234) #oracle32 evo fold 3
input_shape = (28, 28, 1)
conv1 = 137
conv2 = 74
cov3 =49
l_rate =5
kernel1 = 3
kernel2=9
kernel3 = 15
drop1 = 6
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/evo/fold3/evo_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/evo/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 evo fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.8625 - acc: 0.7000
Epoch 1: val_acc improved from -inf to 0.84796, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold3/cnn_saved_model/weights-improvement--01-0.42.hdf5
766/766 [==============================] - 39s 48ms/step - loss: 0.8625 - acc: 0.7000 - val_loss: 0.4204 - val_acc: 0.8480
Epoch 2/50
766/766 [==============================] - ETA: 0s - loss: 0.3857 - acc: 0.8700
Epoch 2: val_acc improved from 0.84796 to 0.90231, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold3/cnn_saved_model/weights-improvement--02-0.29.hdf5
766/766 [==============================] - 36s 46ms/step - loss: 0.3857 - acc: 0.8700 - val_loss: 0.2886 - val_acc: 0.9023
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.3057 - acc: 0.8984
Epoch 3: val_acc improved from 0.90231 to 0.90892, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold3/cnn_saved

In [ ]:
tf.random.set_seed(1234) #oracle32 evo fold 4
input_shape = (28, 28, 1)
conv1 = 55
conv2 = 93
cov3 =39
l_rate =10
kernel1 = 5
kernel2=3
kernel3 = 13
drop1 = 4
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/evo/fold4/evo_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/evo/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 evo fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


766/766 [==============================] - ETA: 0s - loss: 0.6927 - acc: 0.7624
Epoch 1: val_acc improved from -inf to 0.88395, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold4/cnn_saved_model/weights-improvement--01-0.35.hdf5
766/766 [==============================] - 48s 47ms/step - loss: 0.6927 - acc: 0.7624 - val_loss: 0.3488 - val_acc: 0.8840
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3256 - acc: 0.8922
Epoch 2: val_acc improved from 0.88395 to 0.90525, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold4/cnn_saved_model/weights-improvement--02-0.27.hdf5
766/766 [==============================] - 35s 46ms/step - loss: 0.3258 - acc: 0.8922 - val_loss: 0.2742 - val_acc: 0.9053
Epoch 3/50
765/766 [============================>.] - ETA: 0s - loss: 0.2635 - acc: 0.9136
Epoch 3: val_acc improved from 0.90525 to 0.92472, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold4/cnn_saved_model/weig

In [ ]:
tf.random.set_seed(1234) #oracle32 evo fold 5
input_shape = (28, 28, 1)
conv1 = 125
conv2 = 47
cov3 =62
l_rate =7
kernel1 = 3
kernel2=3
kernel3 = 11
drop1 = 3
drop2 = 6


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/oracle32_new/evo/fold5/evo_Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/oracle32_new/evo/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)
 
history = model.fit(x_train, y_train, #oracle32 evo fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
766/766 [==============================] - ETA: 0s - loss: 0.7234 - acc: 0.7538
Epoch 1: val_acc improved from -inf to 0.89130, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold5/cnn_saved_model/weights-improvement--01-0.34.hdf5
766/766 [==============================] - 36s 44ms/step - loss: 0.7234 - acc: 0.7538 - val_loss: 0.3396 - val_acc: 0.8913
Epoch 2/50
765/766 [============================>.] - ETA: 0s - loss: 0.3227 - acc: 0.8947
Epoch 2: val_acc improved from 0.89130 to 0.91480, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold5/cnn_saved_model/weights-improvement--02-0.25.hdf5
766/766 [==============================] - 30s 40ms/step - loss: 0.3227 - acc: 0.8946 - val_loss: 0.2548 - val_acc: 0.9148
Epoch 3/50
766/766 [==============================] - ETA: 0s - loss: 0.2499 - acc: 0.9176
Epoch 3: val_acc improved from 0.91480 to 0.92251, saving model to /content/drive/MyDrive/experiments/oracle32_new/evo/fold5/cnn_saved